In [1]:
import re
import sys
import logging
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import explode, when, col, split, input_file_name, regexp_replace, udf, lit, monotonically_increasing_id
from pyspark.sql.types import StringType
from awsglue.dynamicframe import DynamicFrame

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.4 
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Session ID: 77f58aff-004b-4554-8e6c-e8d35bc9000f
Applying the following default arguments:
--glue_kernel_version 1.0.4
--enable-glue-datacatalog true
Waiting for session 77f58aff-004b-4554-8e6c-e8d35bc9000f to get into ready status...
Session 77f58aff-004b-4554-8e6c-e8d35bc9000f has been created.



In [2]:
args = getResolvedOptions(sys.argv, [])
glueContext = GlueContext(SparkContext.getOrCreate())
spark = glueContext.spark_session
job = Job(glueContext)
job_name = 'de-4-1-backend'
job.init(job_name, args)

In [3]:
# uuid 추가

import uuid
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StringType
import pandas as pd

@pandas_udf(returnType=StringType(), functionType=PandasUDFType.SCALAR)
def generate_uuid(series: pd.Series) -> pd.Series:
    return series.apply(lambda x: str(uuid.uuid4()).replace("-", ""))

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:394: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.


In [4]:
restaurant_dynamic = glueContext.create_dynamic_frame.from_catalog(
    database='de-4-1-kakao-db',
    table_name='info_type_info'
)

In [5]:
restaurant_df = restaurant_dynamic.toDF()
restaurant_df = restaurant_df.drop('location')
restaurant_df = restaurant_df.dropDuplicates(["kakao_id"])
restaurant_df = restaurant_df.withColumn('uuid', generate_uuid(restaurant_df['kakao_id']))

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.


In [6]:
print(restaurant_df.count())

37080


In [ ]:
restaurant_df = restaurant_df.coalesce(1)

restaurant_df.write.format('csv').option('header', 'true').save('s3://de-4-1-glue-test/backend/restaurant/')

In [6]:
resataurant_df = restaurant_df.na.drop(subset=['uuid'])

In [7]:
resataurant_df = resataurant_df.withColumn('name', regexp_replace("name", ",", " "))

In [7]:
restaurant_df.show(5)

+-----------+-----------------------+------------+---------------+-----------+------+-------+------------+------------------+--------+------------+---------+-----------------------------------+----------+-----------------------------+----------+--------+----+-----+---+--------------------+
|last_update|                   name|english_name|        address|postal_code|  xpos|   ypos|       phone|        open_hours|homepage|introduction| category|                               tags|facilities|                        menus|  kakao_id| keyword|year|month|day|                uuid|
+-----------+-----------------------+------------+---------------+-----------+------+-------+------------+------------------+--------+------------+---------+-----------------------------------+----------+-----------------------------+----------+--------+----+-----+---+--------------------+
|2023.11.08.|일번지정육식당 장한평점|            |      장한로 18|      02644|514554|1128674|02-2212-9926|매일 11:00 ~ 23:00|        |         

In [7]:
# location 추가
cong_dynamic = glueContext.create_dynamic_frame.from_options(
    "s3",
    {'paths': ["s3://de-4-1-glue-test/congestion/"]},
    "csv",
    {"withHeader": True}  
)

In [8]:
cong_df = cong_dynamic.toDF()
# cong_df.show(5)

In [9]:
cong_df = cong_df['location', 'gu_id']

# info

In [13]:
info_dynamic = glueContext.create_dynamic_frame.from_options(
    "s3",
    {'paths': ["s3://de-4-1-glue-test/backend/restaurant_restaurantinfo/"]},
    "csv",
    {"withHeader": True}  
)

info_df = info_dynamic.toDF()

In [10]:
info_column = ['uuid', 'name', 'address', 'phone', 'kakao_id', 'keyword', 'last_update']
info_df = restaurant_df[info_column]
info_df = info_df.withColumn('source', lit('K'))
#info_df = info_df.withColumnRenamed('keyword', 'location')
info_df = info_df.withColumnRenamed('last_update', 'store_info_last_update')
# info_df.show(5)

In [11]:
# gu_id 추가
info_df = info_df.join(cong_df, info_df['keyword'] == cong_df['location'], 'left')
info_df = info_df.drop('location')
# info_df.show(5)

In [19]:
info_df.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- kakao_id: string (nullable = true)
 |-- store_info_last_update: string (nullable = true)
 |-- source: string (nullable = false)
 |-- gu_id: string (nullable = true)


In [38]:
info_df.show(10)

+--------------------+----------------------------+-----------------+-------------+----------+----------------------+------+-----+
|                uuid|                        name|          address|        phone|  kakao_id|store_info_last_update|source|gu_id|
+--------------------+----------------------------+-----------------+-------------+----------+----------------------+------+-----+
|e7681da912d541df9...|                      오랑지|    동교로38길 13|             |1007372949|           2024.01.30.|     K|   21|
|b6e36afda9344430a...|           온달집 홍대직영점|       잔다리로 9|  02-332-4936|1009353804|           2023.11.08.|     K|   21|
|762f8ac76a6b4cceb...|스타일난다 핑크풀카페 홍대점|와우산로29다길 23|070-7663-6909|1010612115|           2023.03.15.|     K|   21|
|df146eb9319b4b438...|                    더파이홀|   연세로5나길 20|010-4258-1181|1011256721|           2023.07.14.|     K|   21|
|cd8224ae907f443c8...|                  쿠시노주방|     동교로 218-1|010-4941-6635|1029469924|           2023.11.28.|     K|   21|
|054b31

In [12]:
info_df = info_df.withColumn("gu_id", when(info_df.keyword == "이태원", 21).otherwise(info_df.gu_id))
info_df = info_df.withColumn("gu_id", when(info_df.keyword == "홍대입구역", 13).otherwise(info_df.gu_id))
info_df = info_df.withColumn("gu_id", when(info_df.keyword == "쌍문동", 10).otherwise(info_df.gu_id))
info_df = info_df.withColumn("gu_id", when(info_df.keyword == "방배역", 15).otherwise(info_df.gu_id))
info_df = info_df.withColumn("gu_id", when(info_df.keyword == "시청광장", 24).otherwise(info_df.gu_id))

In [16]:
info_df = info_df.na.drop(subset=['keyword'])
info_df = info_df.na.drop(subset=['uuid'])

In [ ]:
info_df = info_df.drop('keyword')
info_df = info_df.coalesce(1)

info_df.write.format('csv').option('header', 'true').save('s3://de-4-1-glue-test/backend/restaurant_restaurantinfo/')

## category

In [22]:
cate_split = restaurant_df.withColumn('category', explode(split(restaurant_df['category'], ',')))
category_df = cate_split.select('category').distinct()

In [23]:
category_df = category_df.withColumn('id', monotonically_increasing_id())
category_df = category_df.withColumnRenamed('category', 'name')
# category_df.show(20)

In [ ]:
# num_partitions = 20  
# partitioned_cate = category_df.repartition(num_partitions)
category_df = category_df.coalesce(1)

category_df.write.format('csv').option('header', 'true').save('s3://de-4-1-glue-test/backend/restaurant_category/')

In [16]:
print(category_df.count())

557


In [51]:
# category 중간테이블
category_column = ['category', 'uuid']
info_cate_df = restaurant_df[category_column]
info_cate_df = info_cate_df.withColumnRenamed('uuid', 'restaurantinfo_id')
info_cate_df = info_cate_df.withColumn('name', explode(split(info_cate_df['category'], ',')))

info_cate_df = info_cate_df.join(category_df, info_cate_df['name'] == category_df['name'], 'inner')
info_cate_df = info_cate_df.withColumnRenamed('id', 'category_id')

info_cate_df = info_cate_df.withColumn('id', monotonically_increasing_id())

info_cate_df = info_cate_df.drop('category', 'name')

In [52]:
info_cate_df = info_cate_df.na.drop(subset=['category_id'])
# cate.show()

In [53]:
num_partitions = 20  
partitioned_info_cate = info_cate_df.repartition(num_partitions)

partitioned_info_cate.write.format('csv').option('header', 'true').save('s3://de-4-1-glue-test/backend/restaurant_restaurantinfo_category/')

In [33]:
info_cate_df.printSchema()

root
 |-- restaurantinfo_id: string (nullable = true)
 |-- category_id: long (nullable = true)
 |-- id: long (nullable = false)


### open hours

In [32]:
open_split = restaurant_df.withColumn('opening_hours', explode(split(restaurant_df['open_hours'], ';')))
open_df = open_split.select('opening_hours').distinct()
open_df = open_df.withColumn('id', monotonically_increasing_id())
# open_df.show(20)

In [33]:
num_partitions = 10  
partitioned_open = open_df.repartition(num_partitions)

partitioned_open.write.format('csv').option('header', 'true').save('s3://de-4-1-glue-test/backend/restaurant_openhour/')

In [36]:
open_df.printSchema()

root
 |-- opening_hours: string (nullable = false)
 |-- openhour_id: long (nullable = false)


In [34]:
# open_hour 중간테이블
open_column = ['open_hours', 'uuid']
info_open_df = restaurant_df[open_column]
info_open_df = info_open_df.withColumnRenamed('uuid', 'restaurantinfo_id')
info_open_df = info_open_df.withColumn('opening_hours', explode(split(info_open_df['open_hours'], ';')))

info_open_df = info_open_df.join(open_df, info_open_df['opening_hours'] == open_df['opening_hours'], 'left')
info_open_df = info_open_df.withColumnRenamed('id', 'openhour_id')

info_open_df = info_open_df.withColumn('id', monotonically_increasing_id())

info_open_df = info_open_df.drop('open_hours', 'opening_hours')

In [35]:
info_open_df = info_open_df.na.drop(subset=['openhour_id'])

In [36]:
num_partitions = 20  
partitioned_info_open = info_open_df.repartition(num_partitions)

partitioned_info_open.write.format('csv').option('header', 'true').save('s3://de-4-1-glue-test/backend/restaurant_restaurantinfo_open_hours/')

In [39]:
info_open_df.printSchema()

root
 |-- restaurantinfo_id: string (nullable = true)
 |-- openhour_id: long (nullable = true)
 |-- id: long (nullable = false)


## tags

In [40]:
tags_split = restaurant_df.withColumn('tags', explode(split(restaurant_df['tags'], ';')))
tags_df = tags_split.select('tags').distinct()
tags_df = tags_df.withColumn('id', monotonically_increasing_id())
tags_df = tags_df.withColumnRenamed('tags', 'tag')
# tags_df.show(20)

In [26]:
print(tags_df.count())

3735


In [24]:
tags_df.printSchema()

root
 |-- tag: string (nullable = false)
 |-- id: long (nullable = false)


In [41]:
num_partitions = 20  
partitioned_tags = tags_df.repartition(num_partitions)

partitioned_tags.write.format('csv').option('header', 'true').save('s3://de-4-1-glue-test/backend/restaurant_tags/')

In [45]:
# tags 중간테이블
tags_column = ['tags', 'uuid']
info_tags_df = restaurant_df[tags_column]
info_tags_df = info_tags_df.withColumnRenamed('uuid', 'restaurantinfo_id')
info_tags_df = info_tags_df.withColumn('tag', explode(split(info_tags_df['tags'], ';')))

info_tags_df = info_tags_df.join(tags_df, info_tags_df['tag'] == tags_df['tag'], 'left')
info_tags_df = info_tags_df.withColumnRenamed('id', 'tags_id')

info_tags_df = info_tags_df.withColumn('id', monotonically_increasing_id())

info_tags_df = info_tags_df.drop('tags', 'tag')

In [43]:
info_tags_df = info_tags_df.na.drop(subset=['tags_id'])

In [44]:
num_partitions = 20  
partitioned_info_tags = info_tags_df.repartition(num_partitions)

partitioned_info_tags.write.format('csv').option('header', 'true').save('s3://de-4-1-glue-test/backend/restaurant_restaurantinfo_tags/')

In [30]:
# tags_df.printSchema()

root
 |-- tag: string (nullable = false)
 |-- id: long (nullable = false)


## facilities - XXXXX

In [20]:
fac_split = restaurant_df.withColumn('facilities', explode(split(restaurant_df['facilities'], ';')))
fac_df = fac_split.select('facilities').distinct()
fac_df = fac_df.withColumn('uuid', generate_uuid(fac_df['facilities']))
print(fac_df.count())
fac_df.show(20)

5
+--------------------+--------------------+
|          facilities|                uuid|
+--------------------+--------------------+
|                wifi|7cd86d8e-c7a5-486...|
|        pet_friendly|b7ab920e-aa6f-471...|
|             parking|7e738692-ecff-451...|
|                    |85b3cad8-a247-49a...|
|disabled_accessib...|58fa1265-94d8-461...|
+--------------------+--------------------+


In [21]:
# facilities 중간테이블
fac_column = ['facilities', 'uuid']
info_fac_df = restaurant_df[fac_column]
info_fac_df = info_fac_df.withColumnRenamed('uuid', 'restaurant_uuid')
info_fac_df = info_fac_df.withColumn('facilities', explode(split(info_fac_df['facilities'], ';')))

info_fac_df = info_fac_df.join(fac_df, info_fac_df['facilities'] == fac_df['facilities'], 'left')
info_fac_df = info_fac_df.withColumnRenamed('uuid', 'facilities_uuid')

info_fac_df = info_fac_df.withColumn('uuid', generate_uuid(info_fac_df['restaurant_uuid']))

info_fac_df = info_fac_df.drop('facilities', 'facilities')

In [22]:
print(info_fac_df.count())
info_fac_df.show(5)

643688
+--------------------+--------------------+--------------------+
|     restaurant_uuid|     facilities_uuid|                uuid|
+--------------------+--------------------+--------------------+
|61f9758d-295d-4f9...|2e3f069a-d1d8-48f...|1b01346d-f1e3-451...|
|34c565af-22b6-4f0...|56f9519e-b93b-4f4...|3daa3955-c7c7-48c...|
|05f75e58-c7fd-4bc...|56f9519e-b93b-4f4...|431b51a1-a623-4fa...|
|89ee84a2-7f51-4f0...|56f9519e-b93b-4f4...|0a5e39d5-c7c4-495...|
|ebe498a3-3336-4ed...|56f9519e-b93b-4f4...|992cf118-e5e6-4cc...|
+--------------------+--------------------+--------------------+
only showing top 5 rows


## menu

In [ ]:
# menu_df.show()

In [27]:
menu_df.printSchema()

root
 |-- kakao_id: string (nullable = true)
 |-- menu: string (nullable = true)
 |-- price: string (nullable = true)
 |-- description: string (nullable = true)
 |-- img: string (nullable = true)
 |-- id: long (nullable = false)


In [46]:
menu_column = ['kakao_id', 'menus']
menu_df = restaurant_df[menu_column]
menu_df = menu_df.withColumn('menus', explode(split(menu_df['menus'], ';')))

@udf(StringType())
def extract_menu_name(menu):
    if '(' in menu and ')' in menu:
        menu_info = menu.rsplit('(', 1)[0].strip()
        return menu_info
    else:
        return menu

@udf(StringType())
def extract_price(menu):
    if '(' in menu and ')' in menu:
        price_info_parts = menu.split('(')[-1].split(',')
        try:
            return price_info_parts[0] + price_info_parts[1]
        except IndexError:
            return price_info_parts[0] if price_info_parts else "가격 정보 없음"
    else:
        return None

@udf(StringType())
def extract_description(menu):
    if '(' in menu and ')' in menu:
        parts = menu.split(",")
        try:
            return parts[2] 
        except IndexError:
            return "설명 정보 없음"
    else:
        return None

@udf(StringType())
def extract_image(menu):
    if '(' in menu and ')' in menu:
        parts = menu.split(",")
        try:
            return parts[3] 
        except IndexError:
            return "이미지 정보 없음"
    else:
        return None

menu_df = menu_df.withColumn("menu", extract_menu_name("menus")) \
                 .withColumn("price", extract_price("menus")) \
                 .withColumn("description", extract_description("menus")) \
                 .withColumn("img", extract_image("menus"))

menu_df = menu_df.drop('menus')
menu_df = menu_df.withColumn('id', monotonically_increasing_id())

In [47]:
info_menu = menu_df.select('*')

In [48]:
# menu 중간테이블

info_menu_df = info_menu.join(info_df, info_menu.kakao_id == info_df.kakao_id)
info_menu_df = info_menu_df.select(info_menu.id, info_df.uuid)
info_menu_df = info_menu_df.withColumnRenamed('uuid', 'restaurantinfo_id')
info_menu_df = info_menu_df.withColumnRenamed('id', 'menu_id')
info_menu_df = info_menu_df.withColumn('id', monotonically_increasing_id())

In [49]:
info_menu_df = info_menu_df.na.drop(subset=['menu_id'])

In [37]:
info_menu_df.show(20)

+----------+--------------------+---+
|   menu_id|   restaurantinfo_id| id|
+----------+--------------------+---+
|8590311085|0dcf3605374e4d649...|  0|
|8590311085|c6dbd7236a854ea9b...|  1|
|    162028|d2995e8e187848d3a...|  2|
|    162029|d2995e8e187848d3a...|  3|
|    162030|d2995e8e187848d3a...|  4|
|    162031|d2995e8e187848d3a...|  5|
|    162032|d2995e8e187848d3a...|  6|
|      1319|c666d5a310ba44ab8...|  7|
|      1319|6ec77d84c8534b0c9...|  8|
|      1319|36476f5d347e4265b...|  9|
|      1319|7943514809e34b8b9...| 10|
|      1319|dbbe94b5b48c47588...| 11|
|      1319|e9ca53799d1347bdb...| 12|
|      1319|fa20ba88f79b4e51b...| 13|
|      1319|175272fadc2f4e68b...| 14|
|      1319|8c66719074124c91b...| 15|
|      1319|aec4b45dd98b4566a...| 16|
|      1319|bd5340323cae4accb...| 17|
|      1319|c778e027e9724e738...| 18|
|      1319|1fa175510aa3488ab...| 19|
+----------+--------------------+---+
only showing top 20 rows


In [ ]:
# num_partitions = 10
# partitioned_info_menu = info_menu_df.repartition(num_partitions)
info_menu_df = info_menu_df.coalesce(1)

info_menu_df.write.format('csv').option('header', 'true').save('s3://de-4-1-glue-test/backend/restaurant_restaurantinfo_menu/')

In [34]:
print(info_menu_df.count())

381763628


In [35]:
menu_df.printSchema()

root
 |-- kakao_id: string (nullable = true)
 |-- menu: string (nullable = true)
 |-- price: string (nullable = true)
 |-- description: string (nullable = true)
 |-- img: string (nullable = true)
 |-- id: long (nullable = false)


In [36]:
menu_df = menu_df.drop('kakao_id')

num_partitions = 20  
partitioned_menu = menu_df.repartition(num_partitions)

partitioned_menu.write.format('csv').option('header', 'true').save('s3://de-4-1-glue-test/backend/restaurant_menu/')

# review

In [ ]:
review_dynamic = glueContext.create_dynamic_frame.from_catalog(
    database='de-4-1-kakao-db',
    table_name='info_type_reviews'
)

review_df = review_dynamic.toDF()
# print(review_df.count())
# review_df.show(10)

In [ ]:
review_df.printSchema()

In [ ]:
review_df = review_df.drop('location')
review_df = review_df.withColumn('id', monotonically_increasing_id())

In [ ]:
review_column = ['id', 'kakao_id', 'kakaomap_user_id', 'contents', 'rate', 'created_at']
review_df = review_df[review_column]
review_df = review_df.withColumnRenamed('created_at', 'date')
review_df = review_df.withColumnRenamed('contents', 'comment')
review_df = review_df.withColumnRenamed('kakao_id', 'restaurant')
review_df = review_df.withColumnRenamed('kakaomap_user_id', 'username')
review_df = review_df.withColumn('source', lit('K'))
review_df = review_df.join(info_df, review_df['restaurant'] == info_df['kakao_id'], 'left')
review_df = review_df.drop('location', 'restaurant', 'name', 'address', 'phone', 'store_info_last_update', 'source')
review_df = review_df.withColumn('user', lit('a'))
# review_df.show(10)

In [ ]:
num_partitions = 20  
partitioned_review = review_df.repartition(num_partitions)

partitioned_review.write.format('csv').option('header', 'true').save('s3://de-4-1-glue-test/backend/review/')